In [1]:
import os
import sys
import json
os.path.dirname(sys.executable)
sys.path.append("/kuacc/users/ckoksal20/COMP547Project/SSuperGAN/")
#sys.path.append("/home/ckoksal20/Desktop/COMP547/SSuperGAN/")
import torch
from torch import optim
from torch.utils.data import DataLoader
from data.datasets.random_dataset import RandomDataset
from data.datasets.golden_faces import GoldenFacesDataset
from data.datasets.golden_panels import GoldenPanelsDataset

from training.dcgan_trainer import DCGANTrainer
from training.ssuper_dcgan_trainer import SSuperDCGANTrainer
from utils.config_utils import read_config, Config
from utils.logging_utils import *
from utils.plot_utils import *
from utils import pytorch_util as ptu
from configs.base_config import *

from networks.ssuper_dcgan import SSuperDCGAN
from networks.ssuper_msggan import SSuperMSGGAN

import torch.nn as nn




"""
net = SSuperDCGAN(config.backbone, 
                    latent_dim=config.latent_dim, 
                    embed_dim=config.embed_dim,
                    use_lstm=config.use_lstm,
                    seq_size=config.seq_size,
                    gen_img_size=config.image_dim,
                    lstm_hidden=config.lstm_hidden,
                    lstm_dropout=config.lstm_dropout,
                    fc_hidden_dims=config.fc_hidden_dims,
                    fc_dropout=config.fc_dropout,
                    num_lstm_layers=config.num_lstm_layers,
                    masked_first=config.masked_first,
                    ngpu = config.ngpu,
                    ngf = config.ngf,
                    ndf = config.ndf,
                    nc = config.nc,
                    image_size=config.image_dim).to(ptu.device) 
"""    

Random Seed:  999


'\nnet = SSuperDCGAN(config.backbone, \n                    latent_dim=config.latent_dim, \n                    embed_dim=config.embed_dim,\n                    use_lstm=config.use_lstm,\n                    seq_size=config.seq_size,\n                    gen_img_size=config.image_dim,\n                    lstm_hidden=config.lstm_hidden,\n                    lstm_dropout=config.lstm_dropout,\n                    fc_hidden_dims=config.fc_hidden_dims,\n                    fc_dropout=config.fc_dropout,\n                    num_lstm_layers=config.num_lstm_layers,\n                    masked_first=config.masked_first,\n                    ngpu = config.ngpu,\n                    ngf = config.ngf,\n                    ndf = config.ndf,\n                    nc = config.nc,\n                    image_size=config.image_dim).to(ptu.device) \n'

In [2]:
config = read_config(Config.SSUPER_MSGGAN)

In [ ]:
net = SSuperMSGGAN(config.backbone, 
                    latent_dim=config.latent_dim, 
                    embed_dim=config.embed_dim,
                    use_lstm=config.use_lstm,
                    seq_size=config.seq_size,
                    lstm_hidden=config.lstm_hidden,
                    lstm_dropout=config.lstm_dropout,
                    fc_hidden_dims=config.fc_hidden_dims,
                    fc_dropout=config.fc_dropout,
                    num_lstm_layers=config.num_lstm_layers,
                    masked_first=config.masked_first,
                    depth = config.depth,
                    use_eql = config.use_eql,
                    use_ema = config.use_ema,
                    ema_decay = config.ema_decay).to(ptu.device) 

In [ ]:
data = torch.randn(1,3, 3, 360, 360)

In [ ]:
out,out2 = net(data)

In [ ]:
out[-1].shape

In [ ]:
import os
import sys
import json
os.path.dirname(sys.executable)
sys.path.append("/kuacc/users/ckoksal20/COMP547Project/SSuperGAN/")

import torch
from torch import optim
from torch.utils.data import DataLoader
from data.datasets.random_dataset import RandomDataset
from data.datasets.golden_faces import GoldenFacesDataset
from data.datasets.golden_panels import GoldenPanelsDataset

from training.ssuper_msgan_trainer import SSuperMSGGANTrainer

from utils.config_utils import read_config, Config
from utils.logging_utils import *
from utils.plot_utils import *
from utils import pytorch_util as ptu
from configs.base_config import *

from networks.ssuper_msggan import SSuperMSGGAN

import torch.nn as nn

from functional.losses.msggan_losses import *


def save_best_loss_model(model_name, model, best_loss):
    # print('current best loss: ' + str(best_loss))
    logging.info('Current best loss: ' + str(best_loss))
    torch.save(model, base_dir + 'playground/ssuper_dcgan/weights/' + model_name + ".pth")




def train(data_loader,config,dataset, model_name='ssuper_dcgan',):
    # loading config
    logging.info("[INFO] Initiate training...")

    # creating model and training details

    net = SSuperMSGGAN(config.backbone, 
                    latent_dim=config.latent_dim, 
                    embed_dim=config.embed_dim,
                    use_lstm=config.use_lstm,
                    seq_size=config.seq_size,
                    lstm_hidden=config.lstm_hidden,
                    lstm_dropout=config.lstm_dropout,
                    fc_hidden_dims=config.fc_hidden_dims,
                    fc_dropout=config.fc_dropout,
                    num_lstm_layers=config.num_lstm_layers,
                    masked_first=config.masked_first,
                    depth = config.depth,
                    use_eql = config.use_eql,
                    use_ema = config.use_ema,
                    ema_decay = config.ema_decay).to(ptu.device)  
    
    #print(net)
    



    # Setup Adam optimizers for both G and D

    optimizer_encoder = optim.Adam(net.encoder.parameters(), lr=config.lr)

    gen_params = [{'params': net.gan.gen.style.parameters(), 'lr': config.g_lr * 0.01, 'mult': 0.01},
                  {'params': net.gan.gen.layers.parameters(), 'lr': config.g_lr},
                  {'params': net.gan.gen.rgb_converters.parameters(), 'lr': config.g_lr}]
    gen_optim = torch.optim.Adam(gen_params, config.g_lr,
                              [config.beta_1, config.beta_2])

    dis_optim = torch.optim.Adam(net.gan.dis.parameters(), config.d_lr,
                              [config.beta_1, config.beta_2])


    print("Total epochs ",config.train_epochs)

    loss_name = config.loss_function.lower()

    if loss_name == "hinge":
        loss = HingeGAN
    elif loss_name == "relativistic-hinge":
        loss = RelativisticAverageHingeGAN
    elif loss_name == "standard-gan":
        loss = StandardGAN
    elif loss_name == "lsgan":
        loss = LSGAN
    elif loss_name == "lsgan-sigmoid":
        loss = LSGAN_SIGMOID
    elif loss_name == "wgan-gp":
        loss = WGAN_GP
    else:
        raise Exception("Unknown loss function requested")

    
    # init trainer
    trainer = SSuperMSGGANTrainer(model=net,
                         model_name=model_name,
                         train_loader=data_loader,
                         test_loader=None,
                         criterion= loss(net.gan.dis),
                         epochs=config.train_epochs,
                         optimizer_encoder=optimizer_encoder,
                         optimizer_generator= gen_optim,
                         optimized_discriminator= dis_optim,
                         grad_clip=config.g_clip,
                         best_loss_action=lambda m, l: save_best_loss_model(model_name, m, l),
                         save_dir=base_dir + 'playground/ssuper_msggan',
                         checkpoint_every_epoch=True
                        )



    losses, test_losses = trainer.train_epochs()

    logging.info("[INFO] Completed training!")
    
    #print("Losses ",losses)
    save_training_plot(losses['gen_loss'],
                       losses['disc_loss'],
                       "SSPUPER_MSGGAN  Losses",
                       base_dir + 'playground/ssuper_msggan/' + f'results/{model_name}_plot.png'
                      )
    return net


if __name__ == '__main__':
    ptu.set_gpu_mode(True)
    
    config = read_config(Config.SSUPER_MSGGAN)
    golden_age_config = read_config(Config.GOLDEN_AGE)
    cont_epoch = -1
    cont_model = None # "playground/ssupervae/weights/model-18.pth"
    
    # data = RandomDataset((3, 3, 360, 360), (3, config.image_dim, config.image_dim))
    data = GoldenPanelsDataset(golden_age_config.panel_path,
                               golden_age_config.sequence_path, 
                               golden_age_config.panel_dim,
                               config.image_dim, 
                               augment=False, 
                               mask_val=golden_age_config.mask_val,
                               mask_all=golden_age_config.mask_all,
                               return_mask=golden_age_config.return_mask,
                               train_test_ratio=golden_age_config.train_test_ratio,
                               train_mode=True,
                               limit_size=-1)
    data_loader = DataLoader(data, batch_size=config.batch_size, shuffle=True, num_workers=4)
    
    if config.use_lstm:
        model_name ="lstm_ssuper_msggan_model"
    else:
        model_name ="plain_ssuper_msggan_model"
    
    
    
    model = train(data_loader, config, model_name)
    torch.save(model, base_dir + 'playground/ssuper_msggan/results/' + "ssuper_msggan_model.pth")
        
    
        
        

Loaded pretrained weights for efficientnet-b5


  0%|          | 0/18016 [00:00<?, ?it/s]

DISCRIMINATOE Depth :  5  feature_size :  512
RGB TO FEATURES  ModuleList(
  (0): EqualConv2d(
    (conv): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): EqualConv2d(
    (conv): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (2): EqualConv2d(
    (conv): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (3): EqualConv2d(
    (conv): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
  )
) 

DISC LAYERS :  ModuleList(
  (0): DisGeneralConvBlock(
    (conv_1): EqualConv2d(
      (conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv_2): EqualConv2d(
      (conv): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (downSampler): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (lrelu): LeakyReLU(negative_slope=0.2)
  )
  (1): DisGeneralConvBlock(
    (conv_1): EqualConv2d(
      (conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv_2): EqualConv2d(
      (con

Epoch 0, loss 14702.7666, reconstruction_loss 14697.4727, kl_loss 5.2942, disc_loss 0.1284, gen_loss 1472.8979:   0%|          | 4/18016 [00:03<3:59:04,  1.26it/s]

Z shape torch.Size([4, 512]) X shape  torch.Size([4, 3, 3, 300, 300])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])


...

Z shape torch.Size([4, 512]) X shape  torch.Size([4, 3, 3, 300, 300])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])


Epoch 0, loss 12858.9876, reconstruction_loss 12853.7650, kl_loss 5.2226, disc_loss -127454.1390, gen_loss 170707.0750:   2%|▏         | 332/18016 [01:42<1:29:26,  3.30it/s]

Z shape torch.Size([4, 512]) X shape  torch.Size([4, 3, 3, 300, 300])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])


Epoch 0, loss 12861.9415, reconstruction_loss 12856.7100, kl_loss 5.2315, disc_loss -136933.2157, gen_loss 180946.4568:   2%|▏         | 336/18016 [01:43<1:29:30,  3.29it/s]

Z shape torch.Size([4, 512]) X shape  torch.Size([4, 3, 3, 300, 300])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
Y.shape First torch.Size([4, 256, 64, 64])
Y.shape Second torch.Size([4, 256, 32, 32])
